## Issues
* datepicker does not work on firefox and safari because they do not support the HTML date input field
* can not set disabed state for datepicker
* how to set the number of results

In [1]:
from IPython.display import clear_output
from datetime import datetime
from ipywidgets import DatePicker
import urllib.request
from urllib.parse import quote
from pprint import pprint
import json
import ipywidgets as widgets
from ipywidgets import Layout, Box, HBox
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)



widgetStartTime = DatePicker(description="From:", disabled=False,value=datetime(1970,1,1,0,0,0))
widgetEndTime = DatePicker(description="To:", disabled=False,value=datetime.now())
plotButton=widgets.Button(description="Show available products")
useTimeCheckBox=widgets.Checkbox(
    value=False,
    description='Use date',
    disabled=False
)
ile_wynikpw=widgets.BoundedIntText(
value=100,
min=1,
max=100000,
step=1,
description='Number of results:',
disabled=False
)

dc = DrawControl(circle={},rectangle={},polyline={},polygon={}
                ,marker={'shapeOptions':{'color':'#5300FF'}},edit=False)

mission=widgets.Dropdown(
    options=['All','Envisat','Landsat5','Landsat7','Landsat8','Sentinel1','Sentinel2','Sentinel3'],
    description='Mission:',
    value='All',
    disabled=False,
)

instrument=widgets.Dropdown(
    options=['All','Meris','TM','ETM','OLI','OLI_TIRS','TIRS','SAR','MSI','OLCI','SLSTR','SRAL'],
    description='Instrument:',
    disabled=False,
)

punkty=[]


def handle_draw(self, action, geo_json):
    global punkty
    if action=='created':
        punkty.append(geo_json['geometry']['coordinates'])
    elif action=='deleted':
        punkty.remove(geo_json['geometry']['coordinates']) 
        

dc.on_draw(handle_draw)

m=Map(zoom=1)
m.add_control(dc)
wkt=''
layers=[]
def on_button_clicked(b):
    clear_output()
    global punkty,wkt,m,layers
    print (len(punkty))
    for i in m.layers[1:]:        
        m.remove_layer(i)
    
    if useTimeCheckBox.value:
        datetime
        start=widgetStartTime.value.isoformat()
        end=widgetEndTime.value.isoformat()
        tstr="&startDate=%s&completionDate=%s"%(start,end)
        print(tstr)
    else:
        tstr=''
    if len(punkty)>0:
        wkt='&geometry=MULTIPOINT('
        s=[]
        for punkt in punkty:
            s.append('(%f %f)'%(punkt[0],punkt[1]))
        s=",".join(s)
        wkt+=s+')'        
    else:
        wkt=''
    if mission.value!='All':
        col=mission.value+"/"
    else:
        col=''
    if instrument.value!='All':
        ins='&instrument='+instrument.value
    else:
        ins=''      
    request="finder.eocloud.eu/resto/api/collections/"+col+"search.json?_pretty=true&maxRecords=%d"%ile_wynikpw.value+wkt+tstr
    request+=ins
    request="http://"+quote(request,safe="%/:=&?~#+!$,;'@()*[]")
    print(request)        
    with urllib.request.urlopen(request) as response:
        h=response.read()
    h=json.loads(h)
    g=GeoJSON(data=h,hover_style={'fillColor': 'red'})
    layers.append(g)
    m.add_layer(g)
    for feature in h['features']:
        print(feature['properties']['collection'],
              feature['properties']['instrument'],
              feature['properties']['completionDate'],
               feature['properties']['instrument'],
              feature['properties']['productIdentifier'])

def mission_change(*args):     
    instrument.disabled=False
    if args[0]['new']=='Envisat':
        instrument.options=['MERIS']
    elif args[0]['new']=='Landsat5':
        instrument.options=['TM']
    elif args[0]['new']=='Landsat7':
        instrument.options=['ETM']
    elif args[0]['new']=='Landsat8':
        instrument.options=['All','OLI','OLI_TIRS','TIRS']
    elif args[0]['new']=='Sentinel1':
        instrument.options=['SAR']
    elif args[0]['new']=='Sentinel2':
        instrument.options=['MSI']
    elif args[0]['new']=='Sentinel3':
        instrument.options=['All','OL','SL','SR']
    elif args[0]['new']=='All':
        instrument.options=['All','MERIS','TM','ETM','OLI','OLI_TIRS','TIRS','SAR','MSI','OLCI','SLSTR','SR']
        instrument.value='All'
    
def timeCheck(*args):
    if args[0]['new']:
        print(widgetStartTime.get_state())
        widgetStartTime.disabled=False
        widgetEndTime.disabled=False
    else:
        widgetStartTime.disabled=True
        widgetEndTime.disabled=True
    print(args)
            
#Obsługa eventów
plotButton.on_click(on_button_clicked)
mission.observe(mission_change,names='value')
useTimeCheckBox.observe(timeCheck,names='value')

box_layout = Layout(display='flex',
                    flex_flow='column',                    
                    align_items='center',
                    border='solid',                    
                    width='100%')

items=[HBox([ile_wynikpw]),
       HBox([useTimeCheckBox,widgetStartTime,widgetEndTime]),
       widgets.Label(value="Properties:"),
       HBox([mission,instrument]),
       m,
       plotButton]
box = Box(children=items, layout=box_layout)
box
